# Proyecto: Clasificación de Noticias BBC con Redes Neuronales

Objetivo general

Construir un modelo de red neuronal capaz de clasificar noticias en cinco categorías (business, entertainment, politics, sport, tech) utilizando técnicas de procesamiento de lenguaje natural (NLP) y aprendizaje profundo.

**Objetivos específicos**

Adquirir y preparar un conjunto de datos real con noticias categorizadas.

Preprocesar texto crudo para convertirlo en un formato que pueda ser entendido por una red neuronal.

Codificar las etiquetas de clasificación de texto en valores numéricos.

Vectorizar el texto mediante tokenización y embeddings.

Construir y entrenar una red neuronal usando Keras/TensorFlow.

Evaluar el rendimiento del modelo en datos no vistos.

Analizar los errores y posibles mejoras del modelo de clasificación.

🧩
**🔹 1. Recolección de datos**

Se seleccionó el dataset BBC News Classification, el cual contiene noticias en inglés clasificadas en cinco categorías.

Debido a la indisponibilidad temporal de Kaggle, se propuso una descarga alternativa desde GitHub:


**2. Carga y exploración de datos**

Se utiliza pandas para cargar y visualizar los datos.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample_data/bbc-text.csv")
#print(df.head())
#print(df['category'].value_counts())
df.info()
df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


### **3. Preprocesamiento de texto**

Limpieza básica del texto (minúsculas, eliminación de símbolos).

Eliminación de stopwords si se desea.

Codificación de etiquetas con LabelEncoder:

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['category'])

 **4. Vectorización del texto**

Uso de Tokenizer de Keras para convertir palabras en enteros.

Aplicación de pad_sequences para normalizar la longitud de las noticias.

** ¿Qué es la vectorización del texto?**

La vectorización convierte el texto (cadenas de palabras) en secuencias numéricas que pueden ser interpretadas por un modelo de red neuronal.

Por ejemplo:

Texto original: "England beat Australia"
→ Tokenización → [34, 120, 99]
→ Padding (relleno) → [34, 120, 99, 0, 0, ..., 0]

 Tokenización con Keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])

sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, padding='post', maxlen=200)

**5. Construcción del modelo de red neuronal**

Se diseñó un modelo secuencial sencillo con Embedding, GlobalAveragePooling1D y capas densas:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=200),
    GlobalAveragePooling1D(),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### **6. Entrenamiento y evaluación**

Separación del conjunto de entrenamiento y prueba:

In [ ]:
from sklearn.model_selection import train_test_split

X = padded_sequences
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Entrenamiento

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3448 - loss: 1.5891 - val_accuracy: 0.6112 - val_loss: 1.4798
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7025 - loss: 1.4058 - val_accuracy: 0.7955 - val_loss: 1.1525
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8524 - loss: 1.0058 - val_accuracy: 0.8562 - val_loss: 0.7629
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9213 - loss: 0.5934 - val_accuracy: 0.8921 - val_loss: 0.5054
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9644 - loss: 0.3410 - val_accuracy: 0.9124 - val_loss: 0.3736
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9829 - loss: 0.2049 - val_accuracy: 0.9236 - val_loss: 0.3104
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9877 - loss: 0.1411 - val_accuracy: 0.9326 - val_loss: 0.2673
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9954 - loss: 0.1002 - val_accuracy: 0.9303 - val_l

Evaluación del modelo:

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9452 - loss: 0.2080 
Test Accuracy: 0.9393
